In [1]:
import sys
import pandas as pd
import datetime
import time
import matplotlib.pyplot as plt

#Set number of prolific senders to assess in 2nd and 3rd outputs 
num_prolific_senders = 10

#get input from param
#input_filename = sys.argv[1]
input_filename = "enron-event-history-all.csv" #hardcode for jupyter notebook



In [2]:
def sender_recipient_date_df(target_df, with_date_df):
    working_df=target_df.copy()  # get a working copy
    working_df['DateTime'] = with_date_df['DateTime'] #Add DateTime to Dataframe
    working_df = pd.melt(working_df, id_vars = ['DateTime', 'Sent'], value_name = 'Received') #unpivot recepients into 1 Received column
    working_df.reset_index(inplace=True) 
    working_df.index = working_df.set_index(['DateTime']).index.to_period('M').to_timestamp('M') #convert index to month-end dates
    working_df.drop(['DateTime', 'variable'], axis=1, inplace=True) 
    #working_df.head(10)  
   
    return working_df

In [3]:

def plot_output_by_time(target_df, legend_lst, y_label, title, output_fname):
    plt.figure(figsize=(20,10), dpi=100)
    plt.xlabel('Time by Month')
    plt.ylabel(y_label)
    plt.xticks(rotation=45)
    plt.title(title)
    plt.plot(target_df)
    plt.legend(legend_lst)

    plt.savefig(output_fname, format='png')
    #plt.show()


In [4]:
#### Data loading - START ###

try:
    event_hist_df = pd.read_csv(input_filename, header=None, usecols = [0,1,2,3], 
                            names=['DateTime', 'Msg_ID', 'Sent', 'Received']) 
except NameError:
    print("Filename Not Found.")
except Exception as err:      
    print("Error encountered when reading file.")
    print("Error class is:  ", type(err))
    print("Error message is:", err)    
#event_hist_df.head()

#### Data loading - END ###

In [5]:
#### Data Preparation/Cleaning - START ###

working_df = event_hist_df.copy() # get a working copy

#convert Sent and Received columns to lowercase
working_df['Sent'] = working_df['Sent'].str.lower()  
working_df['Received'] = working_df['Received'].str.lower()

In [6]:
#print("File rowcount:", working_df['Msg_ID'].count())
#print("Duplicate Msg_ID rowcount:", working_df['Msg_ID'].duplicated().sum()) 

working_df = working_df.drop_duplicates(subset=['Msg_ID'], keep='first') #2086 records with duplicate Msg_IDs to remove (assumption to be duplicate messages)
working_df.drop(['Msg_ID'], axis=1, inplace=True) 
#print("New working df rowcount:", working_df['Msg_ID'].count()) 


In [7]:
na_columns=working_df.columns[working_df.isna().any()] #get columns with na
working_df = working_df.dropna(subset=na_columns) #remove rows with missing Sent or Received columns values (assumption to be invalid messages)
#print("New working df rowcount:", working_df['Msg_ID'].count())
#working_df['DateTime'].dtype


In [8]:
working_df['DateTime'] = pd.to_datetime(pd.Series(working_df['DateTime']), unit='ms') #convert unix time to datetime
#working_df['DateTime'].describe()
#working_df.head(10)
Cleaned_df = working_df[~working_df.Sent.isin(['notes','announcements'])] #remove "notes" and "announcements" from sender as assumption they are not person name
#Cleaned_df.head()
#### Data Preparation/Cleaning - END ###

### Output 1 - START

In [9]:
#### Output 1 - START ###

#Create Sent counts df
Sent_df = pd.DataFrame()
Sent_df['sent'] = Cleaned_df['Sent'].groupby(Cleaned_df['Sent']).count()
Sent_df.reset_index(inplace = True)
Sent_df.columns=['person', 'sent']
Sent_df.sort_values(by = 'sent', ascending = False, inplace=True) #sort by sent count descending
#Sent_df.head(10)

In [10]:
#Create Received counts df
Split_Received_df = Cleaned_df['Received'].str.split("|",expand = True) #Split recipients by '|' and expand to additional columns 
Split_Received_df = pd.concat([ Cleaned_df[['Sent']], Split_Received_df], axis = 1)

#Split_Received_df.head(10)

In [11]:
#Create Received counts df
Received_df = pd.melt(Split_Received_df, value_name = 'Received') #unpivot for counting in received column
Received_df = Received_df['Received'].value_counts()
Received_df = pd.DataFrame(Received_df)
Received_df.reset_index(level=0, inplace=True)
Received_df.columns = ['person', 'received']
#Received_df.head(10)

In [12]:
#Merge Send and Receive to 1 df

# Merge received df to sent df (so its still sorted desc by sent) and fill missing(na) with 0
output_1_df = pd.merge(Sent_df, Received_df, on = 'person', how = 'outer').fillna(0)
#output_1_df.head(10)


In [13]:
#Save output 1 to csv file
output_1_df.to_csv('Output_1.csv', index = False)

#### Output 1 - END ###

### Output 2 - START

In [14]:
#### Output 2 - START ###

In [15]:
Prolific_Senders_df = pd.DataFrame()
Prolific_Senders_df['Sent'] = output_1_df['person'].head(num_prolific_senders) #Get most prolific senders
#Prolific_Senders_df.head(10)


In [16]:
working_2_df = sender_recipient_date_df(Split_Received_df, Cleaned_df)
    
#Check the Sender column and filter off non-prolific senders 
working_2_df = working_2_df.loc[working_2_df['Sent'].isin(Prolific_Senders_df['Sent'])]

#Resample to plot by month
working_2_df = working_2_df.groupby(working_2_df['Sent'], as_index=True).resample('M').count() 
working_2_df.drop(['Sent', 'index'], axis=1, inplace=True) 

working_2_df.reset_index(inplace=True) 
working_2_df.columns=['Sender','Date', 'Count']

output_2_df = working_2_df.pivot(index="Date", columns="Sender", values = "Count").fillna(0).astype(int) #Pivot for plotting by senders and time
#output_2_df.head()


In [17]:
 
plot_output_by_time(output_2_df, Prolific_Senders_df['Sent'], 'No. of emails sent', 'No. of emails sent by most prolific senders', 'Output_2.png')

#### Output 2 - END ###

### Output 3 - START

In [18]:
#### Output 3 - START ###

working_3_df = sender_recipient_date_df(Split_Received_df, Cleaned_df)
#working_3_df.head(10)

In [19]:
#Check the receipent column and filter off non-prolific senders 
working_3_df = working_3_df.loc[working_3_df['Received'].isin(Prolific_Senders_df['Sent'])]

#before=Received_df.count()
working_3_df=working_3_df.drop_duplicates()    #Drop Duplicates to get unique contact(s) per month   
#print("before:\n", before, "\nafter:\n", Received_df.count())

working_3_df = working_3_df['Sent'].groupby([working_3_df.index, working_3_df['Received']]).count() # Group by Date and Received
working_3_df = pd.DataFrame(working_3_df)
working_3_df.reset_index(inplace = True) # make indices into columns
working_3_df.columns=['Date','Recipient', 'Count']

#working_3_df.head(10)


In [20]:
output_3_df = working_3_df.pivot(index="Date", columns="Recipient", values = "Count").fillna(0).astype(int) #Pivot for plotting by senders and time
#output_3_df.head()

In [21]:
plot_output_by_time(output_3_df, Prolific_Senders_df['Sent'], 'No. of unique contact(s)', 'No. of unique person(s) who contacted the prolific senders', 'Output_3.png')


#### Output 3 - END ###